In [1]:
%env CUDA_VISIBLE_DEVICES=1
%env TRANSFORMERS_CACHE=/mnt/LLM/hub
%env HF_HOME=/mnt/LLM/hub
%env OMP_NUM_THREADS=16

import os
import sys
sys.path.insert(0, '..')

import time
import random
from tqdm.auto import trange, tqdm
import numpy as np
import ipynbname  # pip install ipynbname

import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers

import ipynbname  # pip install ipynbname
import wandb

from src.aq import QuantizedWeight, QuantizedLinear
from src.modelutils import get_model
from src.datautils import get_loaders


torch.set_num_threads(16)
torch.backends.cudnn.allow_tf32 = False
torch.backends.cuda.matmul.allow_tf32 = False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# wandb.login(relogin=True)

env: CUDA_VISIBLE_DEVICES=1
env: TRANSFORMERS_CACHE=/mnt/LLM/hub
env: HF_HOME=/mnt/LLM/hub
env: OMP_NUM_THREADS=16


/home/jheuristic/anaconda3/lib/python3.9/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
class args:
    base_model = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"
    quant_model = "/extra_disk_1/jheuristic/tinyllama-3t-2x8g8/"
    dtype = 'bfloat16'
    model_seqlen = 1024  # can be 2048 for 1.1B, 4096-8192 for larger models
    device_map = 'auto'
    
    dataset = 'pajama'
    nsamples = 256
    seed = 42
    beam_size = 4
    stochastic_rounding_tau = 0.0
    
    code_lr = 1e-3
    code_lr_plateau_scale = 0.5
    code_betas = (0.0, 0.95)
    delta_decay = 1.0
    codebook_lr = 1e-5
    codebook_betas = (0.9, 0.95)
    codebook_grad_accumulation_steps = 8
    
    
    autocast_dtype = torch.bfloat16  # bfloat16 or None (not using grad scaler!)
    training_dtype = torch.float32
    gradient_checkpointing = False
    devices = [device]

In [3]:
train_data = get_loaders(
    args.dataset,
    nsamples=args.nsamples * 10,
    seed=args.seed,
    model_path=args.base_model,
    seqlen=args.model_seqlen,
)

base_model = get_model(args.base_model, None, args.dtype, args.device_map)
if not args.device_map:
    base_model = base_model.to(device)
    
quantized_model = get_model(args.base_model, args.quant_model, args.dtype, args.device_map)
if not args.device_map:
    quantized_model = quantized_model.to(device)
quantized_model = quantized_model.to(args.training_dtype)

Loading red_pajama from togethercomputer/RedPajama-Data-1T-Sample


/home/jheuristic/anaconda3/lib/python3.9/site-packages/datasets/load.py:1429: FutureWarning: The repository for togethercomputer/RedPajama-Data-1T-Sample contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/togethercomputer/RedPajama-Data-1T-Sample
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Loaded data from pajama; len(data)=2560 sequences
Loading pretrained model ...
Model loaded sucсessfully ...
Initializing model with random weights...
Loading quantized model ...
Model loaded sucсessfully ...


In [4]:
from src.pv_ops import create_dequantized_model
dequantized_model, master_parameters = create_dequantized_model(
    quantized_model, reuse_non_quantized=True, dequantized_dtype=args.autocast_dtype
)
for param in dequantized_model.parameters():
    param.data = param.data.to(args.autocast_dtype)

In [6]:
from src.pv_ops import StraightThroughAdamW

optimizer = StraightThroughAdamW(
    named_dequantized_params=dict(dequantized_model.named_parameters()),
    named_master_params=master_parameters,
    update_codes=dict(lr=args.code_lr, betas=args.code_betas),
    max_code_change_per_step=1e-3,
    beam_size=args.beam_size,
    stochastic_rounding_tau=args.stochastic_rounding_tau,
    dequantized_dtype=torch.float32,
)

In [7]:

if args.gradient_checkpointing:
    quantized_model.gradient_checkpointing_enable()
    quantized_model.enable_input_require_grads()
    for module in quantized_model.modules():
        if isinstance(module, QuantizedLinear):
            module.use_checkpoint = True
    dequantized_model.gradient_checkpointing_enable()
    dequantized_model.enable_input_require_grads()


In [8]:
def _run_one_step(args, base_model, dequantized_model, optimizer, train_data, **kwargs):
    optimizer.zero_grad(set_to_none=True)
    with tqdm(train_data, desc="V step") as progress:

        total_loss = 0.0
        for i, batch in enumerate(progress):
            batch = torch.as_tensor(batch, device=device)
            with torch.no_grad():
                teacher_logits = base_model(batch).logits
            student_logits = dequantized_model(batch).logits  # forward accumulates XTX statistics
            loss = kl_div(student_logits, teacher_logits)
            (loss / len(train_data)).backward()  # backward accumulates gradient
            total_loss = loss.item() / (i + 1) + total_loss * i / (i + 1)
            progress.desc = f"V step: accumulating gradients, loss = {total_loss:.9f}"
            del student_logits, teacher_logits, loss
    optimizer.step(**kwargs)
    optimizer.zero_grad(set_to_none=True)  # reset statistics for the next step
    
    return total_loss


def kl_div(student_hiddens, teacher_hiddens):
    C = student_hiddens.shape[-1]  # num classes
    return F.kl_div(
        input=F.log_softmax(student_hiddens.view(-1, C), dim=-1),
        target=F.log_softmax(teacher_hiddens.view(-1, C), dim=-1),
        log_target=True,
        reduction="batchmean",
    )


In [9]:
eval_data = get_loaders(
    'wikitext2',
    seed=args.seed,
    model_path=args.base_model,
    seqlen=args.model_seqlen,
    eval_mode=True,
)

@torch.inference_mode()
def eval_ppl_naive(model, eval_data):
    eval_inps = [
        eval_data[:, start: start + args.model_seqlen] for start in range(0, eval_data.shape[1], args.model_seqlen)
    ]
    total_tokens = 0
    nlls = []
    for input_ids in tqdm(eval_inps):
        input_ids = input_ids.to(device)
        lm_logits = model(input_ids).logits

        shift_logits = lm_logits[:, :-1, :].contiguous()
        shift_labels = input_ids[:, 1:]
        loss_fct = nn.CrossEntropyLoss()
        loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
        neg_log_likelihood = loss.float() * args.model_seqlen
        nlls.append(neg_log_likelihood)
        total_tokens += shift_labels.numel()
    ppl = torch.exp(torch.stack(nlls).sum() / total_tokens)
    return ppl

Loaded data from wikitext2; len(data)=1 sequences


In [10]:
os.environ["WANDB_NOTEBOOK_NAME"] = os.path.join(os.getcwd(), ipynbname.name() + ".ipynb")
wandb.init(
    project="aqml-full-finetuning",
    config = {
        k: v for k, v
        in args.__dict__.items()
        if not k.startswith("__")
    },
    settings=wandb.Settings(code_dir="."),
    name=ipynbname.name(),
)

wandb: Currently logged in as: denismazur8 (routing-experiments). Use `wandb login --relogin` to force relogin


In [11]:
POINTER = 0
def next_train_data():
    global POINTER
    batch = []
    for i in range(args.nsamples):
        batch.append(train_data[POINTER % len(train_data)])
        POINTER += 1
    return batch

In [12]:
last_ppl_eval_score = None
for i in range(100):
    print("STEP", i)
    if i % 5 == 0:
        ppl_eval_score = eval_ppl_naive(dequantized_model, eval_data).item()
        last_ppl_eval_score = ppl_eval_score
        print("wikitext2 ppl (naive)", ppl_eval_score)

    v_step_train_loss = _run_one_step(args, base_model, dequantized_model, optimizer, next_train_data())
    print("v_step_train_loss", v_step_train_loss)
    state = {
        "ppl_eval_score": ppl_eval_score,
        "v_step_train_loss": v_step_train_loss,
        "step": i,
    }
    wandb.log(state, step=i, commit=True)


STEP 0


  0%|          | 0/334 [00:00<?, ?it/s]

wikitext2 ppl (naive) 18.942625045776367


V step:   0%|          | 0/256 [00:00<?, ?it/s]

model.layers.0.self_attn.q_proj.weight change rate: 6.103515625e-05
model.layers.0.self_attn.k_proj.weight change rate: 0.0
model.layers.0.self_attn.v_proj.weight change rate: 9.918212890625e-05
model.layers.0.self_attn.o_proj.weight change rate: 8.58306884765625e-06
model.layers.0.mlp.gate_proj.weight change rate: 1.3871626833861228e-06
model.layers.0.mlp.up_proj.weight change rate: 0.0
model.layers.0.mlp.down_proj.weight change rate: 0.0
model.layers.1.self_attn.q_proj.weight change rate: 0.0
model.layers.1.self_attn.k_proj.weight change rate: 0.0
model.layers.1.self_attn.v_proj.weight change rate: 1.52587890625e-05
model.layers.1.self_attn.o_proj.weight change rate: 0.0
model.layers.1.mlp.gate_proj.weight change rate: 0.0
model.layers.1.mlp.up_proj.weight change rate: 0.0
model.layers.1.mlp.down_proj.weight change rate: 1.3871626833861228e-06
model.layers.2.self_attn.q_proj.weight change rate: 0.0
model.layers.2.self_attn.k_proj.weight change rate: 0.0
model.layers.2.self_attn.v_pro

...

model.layers.0.self_attn.q_proj.weight change rate: 0.00077056884765625
model.layers.0.self_attn.k_proj.weight change rate: 0.0006256103515625
model.layers.0.self_attn.v_proj.weight change rate: 0.0008544921875
model.layers.0.self_attn.o_proj.weight change rate: 0.0008087158203125
model.layers.0.mlp.gate_proj.weight change rate: 0.0006696527707390487
model.layers.0.mlp.up_proj.weight change rate: 0.0008260553586296737
model.layers.0.mlp.down_proj.weight change rate: 0.0006471113883890212
model.layers.1.self_attn.q_proj.weight change rate: 0.0005626678466796875
model.layers.1.self_attn.k_proj.weight change rate: 0.0008392333984375
model.layers.1.self_attn.v_proj.weight change rate: 0.0007781982421875
model.layers.1.self_attn.o_proj.weight change rate: 0.0007486343383789062
model.layers.1.mlp.gate_proj.weight change rate: 0.0008187727653421462
model.layers.1.mlp.up_proj.weight change rate: 0.0008312572608701885
model.layers.1.mlp.down_proj.weight change rate: 0.0008378462516702712
model.

V step:   0%|          | 0/256 [00:00<?, ?it/s]

model.layers.0.self_attn.q_proj.weight change rate: 0.0006608963012695312
model.layers.0.self_attn.k_proj.weight change rate: 0.0007171630859375
model.layers.0.self_attn.v_proj.weight change rate: 0.00079345703125
model.layers.0.self_attn.o_proj.weight change rate: 0.0008687973022460938
model.layers.0.mlp.gate_proj.weight change rate: 0.0008257086155936122
model.layers.0.mlp.up_proj.weight change rate: 0.0006582086789421737
model.layers.0.mlp.down_proj.weight change rate: 0.0006360140978358686
model.layers.1.self_attn.q_proj.weight change rate: 0.0005435943603515625
model.layers.1.self_attn.k_proj.weight change rate: 0.00072479248046875
model.layers.1.self_attn.v_proj.weight change rate: 0.0008544921875
model.layers.1.self_attn.o_proj.weight change rate: 0.0007181167602539062
model.layers.1.mlp.gate_proj.weight change rate: 0.0008017800282686949
model.layers.1.mlp.up_proj.weight change rate: 0.0006360140978358686
model.layers.1.mlp.down_proj.weight change rate: 0.0006512728868983686
mo

V step:   0%|          | 0/256 [00:00<?, ?it/s]

model.layers.0.self_attn.q_proj.weight change rate: 0.0007658004760742188
model.layers.0.self_attn.k_proj.weight change rate: 0.00070953369140625
model.layers.0.self_attn.v_proj.weight change rate: 0.0008392333984375
model.layers.0.self_attn.o_proj.weight change rate: 0.0008611679077148438
model.layers.0.mlp.gate_proj.weight change rate: 0.0008180792210623622
model.layers.0.mlp.up_proj.weight change rate: 0.0006606362294405699
model.layers.0.mlp.down_proj.weight change rate: 0.000640869140625
model.layers.1.self_attn.q_proj.weight change rate: 0.0008382797241210938
model.layers.1.self_attn.k_proj.weight change rate: 0.0005950927734375
model.layers.1.self_attn.v_proj.weight change rate: 0.00087738037109375
model.layers.1.self_attn.o_proj.weight change rate: 0.0008869171142578125
model.layers.1.mlp.gate_proj.weight change rate: 0.0008114901720546186
model.layers.1.mlp.up_proj.weight change rate: 0.0008371527073904872
model.layers.1.mlp.down_proj.weight change rate: 0.0008298700558952987


In [14]:
ppl_eval_score = eval_ppl_naive(dequantized_model, eval_data).item()
print("wikitext2 ppl (naive)", ppl_eval_score)

  0%|          | 0/334 [00:00<?, ?it/s]

wikitext2 ppl (naive) 13.23874568939209
